# VEM 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Mesh files

In [256]:
from numpy import sqrt
x = [0, 1, 1, 0]
y = [0, 0, 1, 1]

"""
vertices =  np.array([x,y]).T
elements = [0, 1, 2, 4]
boundary = [0, 1, 2, 4]


"""
vertices =  [[0,0], [3,0], [3,2], [1.5,4], [0,4]]
elements = [0, 1, 2, 4, 5]
boundary = [0, 1, 2, 4, 5]


### intialise matrices and functions 

In [257]:
#Caluclate the degrees of freedom.
n_dofs = len(vertices)
n_polys = 3

#Initalise matricies
K = np.zeros((n_dofs,n_dofs))
F = np.zeros(n_dofs)
u = np.zeros(n_dofs)

#Defines polynomial ordering
linear_polynomials = [[0,0], [1,0], [0,1]]

#Unitiy function for vector wraping.
def mod_wrap(x,a):
    return (x-1)%a -2


In [258]:
#Summary statistics
n_sides = len(elements)
area = 1
centroid = [0.5,0.5]
diameter = sqrt(2)


area = 10.5
centroid = [19/14, 38/21]
diameter = 5


### Design B and D matrices

In [259]:
D = np.zeros((n_sides, n_polys))
B = np.zeros((n_polys, n_sides))

D[:,0] = 1
B[0,:] = 1


In [287]:
for vertex_id in range(n_sides):
    vert = vertices[vertex_id]
    prev = vertices[mod_wrap(vertex_id - 1,n_sides)]
    next = vertices[mod_wrap(vertex_id + 1,n_sides)]

    vertex_normal = [next[1] - prev[1], next[0] - prev[0]]
    print(vertex_normal)

    for poly_id in range(1,n_polys):
        poly_degree = linear_polynomials[poly_id]
        monomial_grad = np.divide(poly_degree , diameter)
        D[vertex_id, poly_id] = np.dot(np.subtract(vert,centroid) , poly_degree) / diameter
        B[poly_id, vertex_id] =  0.5 * np.dot(monomial_grad , vertex_normal)

[-4, -1.5]


TypeError: list indices must be integers or slices, not tuple

In [270]:
B*20


array([[20., 20., 20., 20., 20.],
       [-8.,  4.,  8.,  4., -8.],
       [ 6.,  6., -3., -6., -3.]])

In [272]:
B = [[4, 4, 4, 4, 4], 
     [8, 4, 8, 4, -8],
     [-6, -6, 3, 6, 3]]

BB = 0.05*np.array(B)
20*BB
1470*D


array([[1470., -399., -532.],
       [1470.,  483., -532.],
       [1470.,  483.,   56.],
       [1470.,   42.,  644.],
       [1470., -399.,  644.]])

In [285]:
def mod_wrap(x,a):
    return (x-1)%a 

mod_wrap(4,4)

3

In [177]:
np.linalg.inv(G)

array([[ 1.        , -0.06802721, -0.09070295],
       [ 0.        ,  2.38095238,  0.        ],
       [ 0.        ,  0.        ,  2.38095238]])

In [174]:
G = np.array([[1050, 30, 40],
              [0, 441, 0],
              [0, 0, 441]])/1050

## ChatGPT python script

In [276]:
import numpy as np
from scipy.sparse import csr_matrix

def mod_wrap(x, a):
    return (x - 1) % a + 1

def virtual_element(mesh, rhs, boundary_condition):
    n_dofs = len(mesh.vertices)
    n_polys = 3  # Number of polynomials (in this case, linear polynomials)
    K = csr_matrix((n_dofs, n_dofs))  # Stiffness matrix
    F = np.zeros(n_dofs)  # Forcing vector
    u = np.zeros(n_dofs)  # Degrees of freedom of the virtual element solution
    linear_polynomials = [(0, 0), (1, 0), (0, 1)]  # Impose an ordering on the linear polynomials

    for el_id, vert_ids in enumerate(mesh.elements, start=1):
        verts = mesh.vertices[vert_ids]  # Coordinates of the vertices of this element
        n_sides = len(vert_ids)

        # Compute geometric information
        area_components = verts[:, 0] * np.roll(verts[:, 1], -1) - np.roll(verts[:, 0], -1) * verts[:, 1]
        area = 0.5 * np.abs(np.sum(area_components))
        centroid = np.sum((verts + np.roll(verts, -1, axis=0)) * np.tile(area_components, (2, 1)).T, axis=0) / (6 * area)
        diameter = 0

        # Compute the diameter by looking at every pair of vertices
        for i in range(n_sides - 1):
            for j in range(i + 1, n_sides):
                diameter = max(diameter, np.linalg.norm(verts[i] - verts[j]))

        D = np.zeros((n_sides, n_polys))
        D[:, 0] = 1
        B = np.zeros((n_polys, n_sides))
        B[0, :] = 1 / n_sides

        for vertex_id in range(n_sides):
            vert = verts[vertex_id]  # This vertex and its neighbours
            prev = verts[mod_wrap(vertex_id - 1, n_sides)]
            next_ = verts[mod_wrap(vertex_id + 1, n_sides)]
            vertex_normal = [next_[1] - prev[1], prev[0] - next_[0]]  # Average of edge normals

            for poly_id in range(1, n_polys):  # Only need to loop over non-constant polynomials
                poly_degree = np.array(linear_polynomials[poly_id])
                monomial_grad = poly_degree / diameter  # Gradient of a linear is constant
                D[vertex_id, poly_id] = np.dot(vert - centroid, poly_degree) / diameter
                B[poly_id, vertex_id] = 0.5 * np.dot(monomial_grad, vertex_normal)

        # Compute the local Ritz projector to polynomials
        projector = np.linalg.lstsq(B @ D, B, rcond=None)[0]
        stabilising_term = (np.eye(n_sides) - D @ projector).T @ (np.eye(n_sides) - D @ projector)
        G = B @ D
        G[0, :] = 0
        local_stiffness = projector.T @ G @ projector + stabilising_term

        # Copy local to global
        K[np.ix_(vert_ids, vert_ids)] += local_stiffness

        # Compute the right-hand side contribution
        F[vert_ids] += rhs(centroid) * area / n_sides

    # Apply boundary conditions
    boundary_vals = boundary_condition(mesh.vertices[mesh.boundary])
    internal_dofs = np.setdiff1d(np.arange(n_dofs), mesh.boundary)
    F[internal_dofs] -= K[internal_dofs, :][:, mesh.boundary] @ boundary_vals

    # Solve the linear system for internal degrees of freedom
    u[internal_dofs] = np.linalg.solve(K[internal_dofs, :][:, internal_dofs], F[internal_dofs])

    # Set boundary values
    u[mesh.boundary] = boundary_vals

    return u

In [279]:
from numpy import sqrt
x = [0, 1, 1, 0]
y = [0, 0, 1, 1]

"""
vertices =  np.array([x,y]).T
elements = [0, 1, 2, 4]
boundary = [0, 1, 2, 4]


"""
vertices =  [[0,0], [3,0], [3,2], [1.5,4], [0,4]]
elements = [0, 1, 2, 4, 5]
boundary = [0, 1, 2, 4, 5]


square = pd.DataFrame({"vertices":vertices, "elements":elements, "boundary":boundary})


In [305]:
def mod_wrap(x, a):
    return (x - 1) % a + 1

def compute_B_and_D(mesh):
    n_polys = 3  # Number of polynomials (in this case, linear polynomials)
    linear_polynomials = [(0, 0), (1, 0), (0, 1)]  # Impose an ordering on the linear polynomials
    B_list = []
    D_list = []

    for vert_ids in mesh['elements']:
        verts = np.array(mesh.vertices[vert_ids] )  # Coordinates of the vertices of this element
        n_sides = len(vert_ids)

        # Compute geometric information
        area_components = verts[:, 0] * np.roll(verts[:, 1], -1) - np.roll(verts[:, 0], -1) * verts[:, 1]
        area = 0.5 * np.abs(np.sum(area_components))
        centroid = np.sum((verts + np.roll(verts, -1, axis=0)) * np.tile(area_components, (2, 1)).T, axis=0) / (6 * area)
        diameter = 0

        # Compute the diameter by looking at every pair of vertices
        for i in range(n_sides - 1):
            for j in range(i + 1, n_sides):
                diameter = max(diameter, np.linalg.norm(verts[i] - verts[j]))

        D = np.zeros((n_sides, n_polys))
        D[:, 0] = 1
        B = np.zeros((n_polys, n_sides))
        B[0, :] = 1 / n_sides

        for vertex_id in range(n_sides):
            vert = verts[vertex_id]  # This vertex and its neighbours
            prev = verts[mod_wrap(vertex_id - 1, n_sides)]
            next_ = verts[mod_wrap(vertex_id + 1, n_sides)]
            vertex_normal = [next_[1] - prev[1], prev[0] - next_[0]]  # Average of edge normals

            for poly_id in range(1, n_polys):  # Only need to loop over non-constant polynomials
                poly_degree = np.array(linear_polynomials[poly_id])
                monomial_grad = poly_degree / diameter  # Gradient of a linear is constant
                D[vertex_id, poly_id] = np.dot(vert - centroid, poly_degree) / diameter
                B[poly_id, vertex_id] = 0.5 * np.dot(monomial_grad, vertex_normal)

        B_list.append(B)
        D_list.append(D)

    return B_list, D_list
 

# Define mesh vertices, elements, and boundary
vertices = [[0, 0], [3, 0], [3, 2], [1.5, 4], [0, 4]]
elements = [[0, 1, 2, 4, 5],]
boundary = [0, 1, 2, 4, 5]

# Create DataFrame for mesh data
mesh = np.array({"vertices": vertices, "elements": elements, "boundary": boundary})

# Compute B and D matrices
B_matrices, D_matrices = compute_B_and_D(mesh)

# Print the first B and D matrices
print("B matrix:")
print(B_matrices[0])
print("D matrix:")
print(D_matrices[0])


ValueError: All arrays must be of the same length

In [310]:
mesh = np.array({"vertices": vertices, "elements": elements, "boundary": boundary})
mesh

array({'vertices': [[0, 0], [3, 0], [3, 2], [1.5, 4], [0, 4]], 'elements': [[0, 1, 2, 4, 5]], 'boundary': [0, 1, 2, 4, 5]},
      dtype=object)